In [15]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

#import textblob
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import numpy as np
import pandas as pd
import string

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/nimori/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# create a dataframe using texts and lables

df= pd.read_csv('data_1000.csv')
df2= pd.DataFrame()
df2['text']= df['post_description_en']
df2['labels']= df['categories']

In [4]:
df2['labels']= df2['labels'].replace('Journalist, Writing And Translation', 'Government Works')

In [5]:
df2.groupby('labels').count()

,text
labels,
Agriculture And Farming,1
Artist,3
Brokering / Agent,1
Computer Phone Internet,51
Education And Skill Development,142
Fisheries And Animal Husbandry,13
Forest & Environment,15
Government Works,298
Healthcare And Medicine,391


In [6]:
df2= df2[(df2.labels!='Agriculture And Farming')& (df2.labels!='Artist')& (df2.labels!='Brokering / Agent')& (df2.labels!='Labor')& (df2.labels!='Plumber')& (df2.labels!='Sanitation Works')& (df2.labels!='Travel And Transport')]
#We remove all those categories whose count is in single digits

In [7]:
df2.groupby('labels').count()

,text
labels,
Computer Phone Internet,51
Education And Skill Development,142
Fisheries And Animal Husbandry,13
Forest & Environment,15
Government Works,298
Healthcare And Medicine,391
Social Service,74


In [9]:
df3= pd.read_csv('report_04-11.csv')
#len(df3)

In [10]:
df_04= pd.DataFrame()
df_04['text']= df3['translation']
df_04['labels']= df3['classifier']

In [11]:
df_04['labels']= df_04['labels'].apply(lambda x: x.lower())
df_04['text']= df_04['text'].apply(lambda x: x.lower())

In [13]:
def text_cleaning(a):
    list2 = nltk.word_tokenize(a)  
    a_lemma = ' '.join([lemmatizer.lemmatize(words) for words in list2])
    rem_punc= [char for char in a_lemma if char not in string.punctuation]
    rem_punc= ''.join(rem_punc)
    return [word for word in rem_punc.split() if word.lower() not in stopwords.words('english')]

In [17]:
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /home/nimori/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [19]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

stopWords= set(stopwords.words('english'))

In [20]:
df_04['text_cleaned']= df_04.iloc[:,0].apply(text_cleaning)

In [21]:
for i,row in df_04.iterrows():
    row['text_cleaned']= ' '.join([str(elem) for elem in row['text_cleaned']])

In [22]:
df_04

,text,labels,text_cleaned
0,.@ceonagaland briefs media after closing of b...,computer phone internet,ceonagaland brief medium closing byeelection d...
1,patrolling on river ganga and gandak at sonpur...,fisheries and animal husbandry,patrolling river ganga gandak sonpur alectiond...
2,"shri vinay kumar tripathi, general manager, no...",safety and security,shri vinay kumar tripathi general manager nort...
3,the bye-election to the 14 southern angami-i a...,politics,byeelection 14 southern angamii ac wa conducte...
4,"voters in samastipur, playing their part in de...",government works,voter samastipur playing part democracy differ...
...,...,...,...
99,"when shopping in the market, follow all necess...",safety and security,shopping market follow necessary safety measur...
100,online application for admission in sainik sch...,education and skill development,online application admission sainik school amb...
101,"the 12th episode of lokvani, the monthly radio...",education and skill development,12th episode lokvani monthly radio talk chief ...
102,haryana government @mlkhattar has sanctioned r...,agriculture and farming,haryana government mlkhattar ha sanctioned r 4...


In [23]:
df2['text_cleaned']= df2.iloc[:,0].apply(text_cleaning)

In [24]:
for i,row in df2.iterrows():
    row['text_cleaned']= ' '.join([str(elem) for elem in row['text_cleaned']])

In [25]:
df2['labels']= df2['labels'].apply(lambda x: x.lower())
df2['text']= df2['text'].apply(lambda x: x.lower())
df2['text_cleaned']= df2['text_cleaned'].apply(lambda x: x.lower())

In [26]:
df2

,text,labels,text_cleaned
0,#covid19 report of #solapur city. @mahadgipr @...,government works,covid19 report solapur city mahadgipr infodivp...
1,📚 dastan-e-raza library 📚\n \npage 17\n\nastro...,social service,📚 dastaneraza library 📚 page 17 astronomical i...
2,job seekers register now at https://t.co/der9r...,government works,job seeker register http tcoder9rdaue5 employm...
3,# covid19 report of #solapur district. @mahadg...,government works,covid19 report solapur district mahadgipr maha...
4,@15thfincom headed by chairman @nksingh_mp con...,healthcare and medicine,15thfincom headed chairman nksinghmp concludes...
...,...,...,...
995,keshubhai patel was a popular leader and skill...,education and skill development,keshubhai patel wa popular leader skilled orga...
996,parks etc. have also been opened to the public...,computer phone internet,parks etc also opened public unlock 5 situatio...
997,#worldstrokeday2020 \n#smoking #tobacco increa...,government works,worldstrokeday2020 smoking tobacco increase ri...
998,on behalf of the rural development and panchay...,education and skill development,behalf rural development panchayat department ...


In [28]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df2['text_cleaned'], df2['labels'], test_size=0.25, random_state=0, stratify= df2['labels'])
from sklearn.model_selection import StratifiedShuffleSplit
# sss = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
# sss.get_n_splits(X, y)
# for train_index, test_index in sss.split(X, y):
#     #print("TRAIN:", train_index, "TEST:", test_index)
#     train_x, valid_x = X[train_index], X[test_index]
#     train_y, valid_y = y[train_index], y[test_index]

In [29]:
x_test= df_04.text_cleaned
y_test= df_04.labels

In [30]:
# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)
y_test = encoder.fit_transform(y_test)

In [31]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
#count_vect.fit(df2['text_cleaned'])
count_vect.fit(train_x)

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)
xtest_count =  count_vect.transform(x_test)

In [32]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(df2['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)
xtest_tfidf =  tfidf_vect.transform(x_test)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(df2['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)
xtest_tfidf_ngram =  tfidf_vect_ngram.transform(x_test)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(df2['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 
xtest_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(x_test)

/home/nimori/venv_envs/tf24/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:506: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [36]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(df2['text'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [37]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [38]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y, xvalid_count)
print("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy)

NB, Count Vectors:  0.6341463414634146
NB, WordLevel TF-IDF:  0.5934959349593496
NB, N-Gram Vectors:  0.5813008130081301
NB, CharLevel Vectors:  0.491869918699187


In [39]:
# Linear Classifier on Count Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_count, train_y, xvalid_count)
print ("LR, Count Vectors: ", accuracy)

# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy)

LR, Count Vectors:  0.6626016260162602
LR, WordLevel TF-IDF:  0.6219512195121951
LR, N-Gram Vectors:  0.5813008130081301
LR, CharLevel Vectors:  0.6463414634146342


In [40]:
# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print("SVM, N-Gram Vectors: ", accuracy)

SVM, N-Gram Vectors:  0.5772357723577236


In [41]:
# RF on Count Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count)
print("RF, Count Vectors: ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print("RF, WordLevel TF-IDF: ", accuracy)

RF, Count Vectors:  0.6382113821138211
RF, WordLevel TF-IDF:  0.6504065040650406
